In [2]:
import pandas as pd

In [3]:
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


In [48]:
import snowflake.connector

# Establecer la conexión
conn = snowflake.connector.connect(
    user='Datanomads',
    password='Universidad12345.',
    account='my62916.mexico-central.azure',  # ejemplo: ab12345.us-central1.gcp
    warehouse='COMPUTE_WH',
    database='HACKADATA',
    schema='SILVER',  # Opcional
)

# Crear un cursor
cursor = conn.cursor()

# Ejecutar una consulta de ejemplo
cursor.execute("SELECT CURRENT_VERSION()")

# Obtener resultados
for row in cursor:
    print(row)


('9.11.2',)


In [49]:
from datetime import datetime

# Inicio: 2024-05-01 14:00:00
dt_inicio = datetime(2024, 5, 1, 14, 0, 0)

# Fin:    2024-05-01 15:00:00
dt_fin    = datetime(2024, 5, 1, 15, 0, 0)# :contentReference[oaicite:1]{index=1}


In [50]:
sql = """
SELECT *
FROM IA_DATA
WHERE created_at BETWEEN %s AND %s
"""

# Ejecuta la consulta pasando los parámetros
cursor.execute(sql, (dt_inicio, dt_fin))

In [51]:
# Si quieres un cursor normal:
rows = cursor.fetchall()

# O, para un DataFrame de pandas (más cómodo para análisis):
df = cursor.fetch_pandas_all()

df.head()

,ACTOR_ID,CREATED_AT,ID,REPO_ID,TYPE,PESO
0,4553345,2024-05-01 15:00:00,37993503940,42528680,PullRequestReviewEvent,3
1,3603796,2024-05-01 15:00:00,37993503956,724082501,PushEvent,3
2,22429695,2024-05-01 15:00:00,37993503958,495730312,IssueCommentEvent,1
3,164721811,2024-05-01 15:00:00,37993503976,794600379,CreateEvent,2
4,49092133,2024-05-01 15:00:00,37993503984,792877994,PushEvent,3


In [23]:
unique_users = df["ACTOR_ID"].unique()
unique_repos = df["REPO_ID"].unique()

user2idx = {u: i for i, u in enumerate(unique_users)}
repo2idx = {r: i for i, r in enumerate(unique_repos)}

In [52]:
interactions = (
    df
    .groupby(["ACTOR_ID", "REPO_ID"])["PESO"]
    .sum()
    .reset_index()
)

In [54]:
from scipy.sparse import coo_matrix

row = interactions["ACTOR_ID"].map(user2idx)
col = interactions["REPO_ID"].map(repo2idx)
data = interactions["PESO"]

R = coo_matrix(
    (data, (row, col)),
    shape=(len(unique_users), len(unique_repos))
).tocsr()

In [30]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 63.7 MB/s eta 0:00:00


In [ ]:
from implicit.als import AlternatingLeastSquares

# Transponer porque implicit espera matriz item×user
model = AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=20
)
model.fit(R.T)


/usr/local/lib/python3.11/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.002044677734375 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [35]:
# Matriz usuario×repositorio (CSR)
user_items = R.tocsr()

In [44]:
model.recommend(546, user_items[546], N=3)

(array([12049, 10123,   487], dtype=int32),
 array([0.0001859 , 0.00016831, 0.00016317], dtype=float32))